# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Импортирование модулей**

In [2]:
import pandas as pd                                  # для выгрузки данных
import numpy as np
import os

from sklearn.model_selection import train_test_split # для разбиения выборок
from sklearn.preprocessing import StandardScaler     # для масштабирования

from sklearn.linear_model import LinearRegression    # модель, которая будет обучаться

from sklearn.metrics import mean_squared_error

from typing import Union
from itertools import chain

## Загрузка и подготовка данных

In [4]:
os.chdir(r'C:\Users\Grine\Desktop\GitHub\YandexPracticum\6 МАШИННОЕ ОБУЧЕНИЕ В БИЗНЕСЕ\\')

In [6]:
# загрузим исходные данные
data0 = pd.read_csv('geo_data_0.csv')
data1 = pd.read_csv('geo_data_1.csv')
data2 = pd.read_csv('geo_data_2.csv')

In [7]:
#изучим данные:
for i in [data0, data1, data2]:
    i.info()
    print('\n'*2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non

**Выводы по данным:**

    1) Пропуски в данных отсутсвуют;
    2) f0, f1 и f2, признаки, для обучения модели;
    3) product - целевой признак, который необходимо спрогнозировать;
    4) столбец id - уникальный идентификатор, он не поможет в процессе обучения. От него можно    избавиться;
    5) Необходимо оценить масштаб признаков;

In [8]:
#Удалим не нужные поля и создадим список дата-фреймов
data_all = [data0.drop('id', axis=1), data1.drop('id', axis=1), data2.drop('id', axis=1)] 

In [9]:
# посмотрим корреляцию данных для каждого датасета
[display(data.corr().round(5).head(3)) for data in data_all];

,f0,f1,f2,product
f0,1.00000,-0.44072,-0.00315,0.14354
f1,-0.44072,1.00000,0.00172,-0.19236
f2,-0.00315,0.00172,1.00000,0.48366


,f0,f1,f2,product
f0,1.00000,0.18229,-0.00178,-0.03049
f1,0.18229,1.00000,-0.00260,-0.01015
f2,-0.00178,-0.00260,1.00000,0.99940


,f0,f1,f2,product
f0,1.00000,0.00053,-0.00045,-0.00199
f1,0.00053,1.00000,0.00078,-0.00101
f2,-0.00045,0.00078,1.00000,0.44587


**Видно, что признак f2 для второго дата-сета имеет практическую 100% корреляцию с целевым признаком. Это говорит о том, что результат линейной регрессии для второй модели, вероятнее всего будет наиболее точным.** 

In [10]:
# дополнительно изучим, необходимо ли создавать логику 
[display(data.describe().round(2)) for data in data_all];

,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,0.50,0.25,2.50,92.50
std,0.87,0.50,3.25,44.29
min,-1.41,-0.85,-12.09,0.00
25%,-0.07,-0.20,0.29,56.50
50%,0.50,0.25,2.52,91.85
75%,1.07,0.70,4.72,128.56
max,2.36,1.34,16.00,185.36


,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,1.14,-4.80,2.49,68.83
std,8.97,5.12,1.70,45.94
min,-31.61,-26.36,-0.02,0.00
25%,-6.30,-8.27,1.00,26.95
50%,1.15,-4.81,2.01,57.09
75%,8.62,-1.33,4.00,107.81
max,29.42,18.73,5.02,137.95


,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.60
max,7.24,7.84,16.74,190.03


**Поскольку мы видим, что во всех 3 датафреймах, стандартное отклонение разное, как и среднее значение, как и кол-во нефти в запасах, кажется, что стоит масштабировать данные, чтобы повысить точность потенциальной аналитики.**

In [11]:
features_train_all = [] # создадим список всех выборок с параметрами для обучения
features_valid_all = [] # создадим список всех выборок с параметрами для оценки результатов

target_train_all = [] # создадим список всех выборок с целевым параметром для обучения
target_valid_all = [] # создадим список всех выборок с целевым параметром для оценки результатов

# пройдём циклом по всем данным и отберём нужные данные, заполним списки.
for data in data_all:
    features_train, features_valid, target_train, target_valid = train_test_split(data.iloc[:, :-1],
                                                                                  data.iloc[:, -1:],
                                                                                  test_size=0.25,
                                                                                  random_state=12345) 
    features_train_all.append(features_train)
    features_valid_all.append(features_valid)
    target_train_all.append(target_train.values)
    target_valid_all.append(target_valid.values)

In [12]:
# пройдёмся циклом по всем параметрам и стандартизируем их
for i, features_train_data in enumerate(features_train_all):
    scaler = StandardScaler() # вызовем модель
    scaler.fit(features_train_data) # обучим её на тестовых данных
    features_train_all[i] = scaler.transform(features_train_data) # преобразуем данные в списке
    features_valid_all[i] = scaler.transform(features_valid_all[i]) # преобразуем данные в списке

**Перед масштабированием было принято решение разбить данные на обучающие и валидационную выборки**

## Обучение и проверка модели

In [13]:
def create_true_list(x: Union[np.ndarray, list, pd.core.frame.DataFrame],
                     y: Union[np.ndarray, list, pd.core.frame.DataFrame]) -> list:
    '''
    Функция получает на входе 2 массива, списка или датафрейма. x-массив является предсказанием, а 
    y-массив - целевым параметром. Важно, чтобы у объектов x и y был схожий размер. 
    ------
    На выходе, функция возвращает список из тех значений, которые совпали в переменное x и y.
    '''  
    variable = x.tolist() 
    return list(chain.from_iterable([variable[i] for i, (a, b) in enumerate(zip(x, y))
                                if int(a) == int(b)]))

In [14]:
predicted_all = [] # переменная, куда будут записаны все предсказания линейной регресии
true_results = []  # переменная, в которую будут отложены только правильно-предсказанные значения

for i in range(3):
    model = LinearRegression()                            # обратимся к моделе
    model.fit(features_train_all[i], target_train_all[i]) # поочередно обучим её
    predict = model.predict(features_valid_all[i])        # предскажем запасы
    predicted_all.append(predict)
    true_results.append(create_true_list(predict, target_valid_all[i]))
    print(f'Для месторождения {i+1}, модель в среднем предсказала {int(predict.mean())} баррелей нефти')
    print(f'Для модели {i+1} RMSE:', mean_squared_error(predict, target_valid_all[i])**0.5)
    print(f'Всего правильных предсказаний {round(len(true_results[i])/len(target_valid_all[i]), 2)}%')
    print('\n')
    

Для месторождения 1, модель в среднем предсказала 92 баррелей нефти
Для модели 1 RMSE: 37.5794217150813
Всего правильных предсказаний 0.01%


Для месторождения 2, модель в среднем предсказала 68 баррелей нефти
Для модели 2 RMSE: 0.893099286775617
Всего правильных предсказаний 0.35%


Для месторождения 3, модель в среднем предсказала 94 баррелей нефти
Для модели 3 RMSE: 40.02970873393434
Всего правильных предсказаний 0.01%




**Во-первых, исходя из результатов аналитики, можно сделать вывод, что действительно точные предсказания (с погрешностью менее 1 барреля запасов, предоставляет 2 модель. Связано это, с тем, что именно во втором случае, мы наблюдаем значительную корреляцию признака f2. Для моделей линейной регресии, такое значение корреляции определяет очень значимый вес аргумента, что позволяет реализовывать успешные прогнозы.**

**Во-вторых, среднее предсказанное значение модели совпало с средним, из исходного набора данных**

**В-третьих, RMSE модели для 1 и 3 модели черезвычайно низкое (подразумевается, что при среднем кол-ве запасов в 90-92 барреля, ошибка на 37-40 у.е. черезвычайно опасна), однако, нам всё же удалось отобрать 200 необходимых (совпавших) значений, для дальнейшего проведения аналитики**



## Подготовка к расчёту прибыли

In [15]:
BUDGET = 10_000_000_000
BARREL_PRICE = 450
PRODUCT_VALUES = 1_000
product_price=BARREL_PRICE * PRODUCT_VALUES

minimal_average_to_explore_region= BUDGET/PRODUCT_VALUES
minimal_average_to_drilling_wells = BUDGET/200/product_price

print('Минимальное  среднее количество продукта в месторождениях региона, достаточное для разработки:',
      int(minimal_average_to_explore_region))

print('Минимальное  среднее количество продукта в месторождение, для ренатбельной добычи:',
      int(minimal_average_to_drilling_wells))

Минимальное  среднее количество продукта в месторождениях региона, достаточное для разработки: 10000000
Минимальное  среднее количество продукта в месторождение, для ренатбельной добычи: 111


**Для того, чтобы приступить к добычи, в месторождение региона необходимо, чтобы было 22222 баррелей**
**Чтобы одно, случайно-взятое месторождение было рентабельным его запасы должны составлять 111 баррелей**

In [16]:
def revenue(t, p,
            pp=product_price,
            count=200):
    '''
    t - реальные запасы на месторождение
    p - массив из предсказаний запасов на месторождении
    product_price - цена за n-ый объём баррелей
        ____
    Функция рассчитывает выручку для топ-200 месторождений
    '''
    sorted_p = p.sort_values(by='filter', ascending=False)
    selected_data = t.loc[sorted_p.index][:count]
    return selected_data.sum().values*pp - int(BUDGET)

## Расчёт прибыли и рисков 

In [17]:
state = np.random.RandomState(12345)

values = []
for i in range(3):
    predict_to_subsample = pd.DataFrame(predicted_all[i])
    target_to_subsample = pd.DataFrame(target_valid_all[i])
    means = []
    values.append(means)
    for _ in range(1000):
        predict_subsample = predict_to_subsample.sample(n=500,replace=True,random_state=state)
        target_subsample = target_to_subsample.loc[predict_subsample.index]
        predict_subsample.columns = ['filter']
        target_subsample.columns = ['filter']
        means.append(revenue(target_subsample, predict_subsample))

    print(f'Исследование метрик для региона №{i+1}')
    print('-----------------------------------------')
    print(f'Средняя прибыль для региона составляет:', int(*pd.Series(values[i]).mean()))
    print(f'0.025 доверительный интервал по прибыли для региона составляет:', int(*pd.Series(values[i]).quantile(0.025)))
    print(f'0.975 доверительный интервал по прибыли для региона составляет:', int(*pd.Series(values[i]).quantile(0.975)))
    print(f'Риск убытков для региона составляет:', (len([means for means in values[i] if means < 0])/len(values[i]))*100, '\n'*2)


Исследование метрик для региона №1
-----------------------------------------
Средняя прибыль для региона составляет: 425938526
0.025 доверительный интервал по прибыли для региона составляет: -102090094
0.975 доверительный интервал по прибыли для региона составляет: 947976353
Риск убытков для региона составляет: 6.0 


Исследование метрик для региона №2
-----------------------------------------
Средняя прибыль для региона составляет: 518259493
0.025 доверительный интервал по прибыли для региона составляет: 128123231
0.975 доверительный интервал по прибыли для региона составляет: 953612982
Риск убытков для региона составляет: 0.3 


Исследование метрик для региона №3
-----------------------------------------
Средняя прибыль для региона составляет: 420194005
0.025 доверительный интервал по прибыли для региона составляет: -115852609
0.975 доверительный интервал по прибыли для региона составляет: 989629939
Риск убытков для региона составляет: 6.2 




В результате исследования было установлено, что наиболее потенциально прибыльным регионом является регион под номером - 2.
Для этого региона характерно:
    
    1) Средняя потенциальная прибыль в размере: 518.259.493 у.е.
    2) Нижняя граница доверительного интервала: 128.123.231 у.е.
    3) Верхняя граница доверительного интервала: 953.612.982 у.е.
    4) Риск убытков: 0.3%
    
 При этом, стоит отметить, что работа происходила с разными статистическими выборками. Для второго региона наблюдался признак, которые имеет корреляцию стремяющуюясь к единицы. В связи с этим, моделью было отобрано большее кол-во месторождений. Тем не менее это не мешает сделать вывод о том, что именно второй регион является наиболее интересным для заказчика.